In [76]:
# Import Libraries

import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!python3 -m pip install gensim

In [2]:
corpus = [
  'Text of the first document.',
  'Text of the second document made longer.',
  'Number three.',
  'This is number four.',
]

In [3]:
from gensim.utils import simple_preprocess
import gensim

/home/ubuntu/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
tokenized_sentences = [simple_preprocess(sentence) for sentence in corpus]

In [5]:
len(tokenized_sentences)

4

In [6]:
tokenized_sentences

[['text', 'of', 'the', 'first', 'document'],
 ['text', 'of', 'the', 'second', 'document', 'made', 'longer'],
 ['number', 'three'],
 ['this', 'is', 'number', 'four']]

In [7]:
model = gensim.models.Word2Vec(tokenized_sentences, min_count=1, workers=4, window=5)

In [8]:
print(model)

Word2Vec(vocab=13, vector_size=100, alpha=0.025)


In [9]:
print(list(model.wv.key_to_index)[0:100])

['number', 'document', 'the', 'of', 'text', 'four', 'is', 'this', 'three', 'longer', 'made', 'second', 'first']


In [10]:
print(list(model.wv.index_to_key)[0:100])

['number', 'document', 'the', 'of', 'text', 'four', 'is', 'this', 'three', 'longer', 'made', 'second', 'first']


In [11]:
print(model.wv['three'])

[-9.5785474e-03  8.9431144e-03  4.1650678e-03  9.2347339e-03
  6.6435025e-03  2.9247357e-03  9.8040197e-03 -4.4246409e-03
 -6.8033123e-03  4.2273807e-03  3.7290000e-03 -5.6646108e-03
  9.7047593e-03 -3.5583067e-03  9.5494054e-03  8.3472492e-04
 -6.3384580e-03 -1.9771170e-03 -7.3770545e-03 -2.9795242e-03
  1.0416961e-03  9.4826864e-03  9.3558477e-03 -6.5958784e-03
  3.4751510e-03  2.2755694e-03 -2.4893521e-03 -9.2291739e-03
  1.0271263e-03 -8.1657078e-03  6.3201878e-03 -5.8000805e-03
  5.5354382e-03  9.8337224e-03 -1.6000033e-04  4.5284913e-03
 -1.8094016e-03  7.3607611e-03  3.9400961e-03 -9.0103243e-03
 -2.3985051e-03  3.6287690e-03 -9.9568366e-05 -1.2012720e-03
 -1.0554385e-03 -1.6716027e-03  6.0495140e-04  4.1650939e-03
 -4.2527914e-03 -3.8336229e-03 -5.2816868e-05  2.6935578e-04
 -1.6880751e-04 -4.7855065e-03  4.3134023e-03 -2.1719194e-03
  2.1035385e-03  6.6652300e-04  5.9696771e-03 -6.8423818e-03
 -6.8157101e-03 -4.4762585e-03  9.4358278e-03 -1.5918827e-03
 -9.4292425e-03 -5.45041

In [101]:
import pandas as pd
import numpy as np

data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [102]:
# data_1 = pd.read_csv("ner_dataset.csv", encoding="latin1")
# data_1 = data_1.fillna(method="ffill")
# data_1.tail(10)

In [103]:
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [104]:
data.head(40)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [105]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); 
n_words

35179

In [106]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

In [17]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [18]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [19]:
sentences = getter.sentences

In [20]:
sentences[:5]

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Iranian', 'JJ', 'B-gpe'),
  ('officials', 'NNS', 'O'),
  ('say', 'VBP', 'O'),
  ('they', 'PRP', 'O'),
  ('expect', 'VBP', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('access', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('sealed', 'JJ', 'O'),
  ('sensitive', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('plant', 'NN', 'O'),
  ('Wednesday', 'NNP', 'B-tim'),
  ('

In [21]:
max_len = 75
max_len_char = 10

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [22]:
print(word2idx["Obama"], tag2idx["B-geo"])

17459 3


In [23]:
from keras.preprocessing.sequence import pad_sequences
X_word = [[word2idx[w[0]] for w in s] for s in sentences]

In [24]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')

In [25]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

98


In [26]:
list(chars)[:5]

[':', 'l', 'j', 'F', 'B']

In [27]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [28]:
X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [29]:
X_char[0].shape

(75, 10)

In [30]:
print(max_len, max_len_char)

75 10


In [31]:
X_char[0]

array([[69, 67, 58, 71, 80, 72, 64, 12, 80,  0],
       [58, 32,  0,  0,  0,  0,  0,  0,  0,  0],
       [12, 53, 35, 58, 64, 80, 66, 94, 72, 66],
       [67, 72, 87, 53,  0,  0,  0,  0,  0,  0],
       [35, 72, 94, 24, 67, 53, 12,  0,  0,  0],
       [66, 67, 94, 58, 71, 68, 67,  0,  0,  0],
       [13, 58, 64, 12, 58, 64,  0,  0,  0,  0],
       [66, 58,  0,  0,  0,  0,  0,  0,  0,  0],
       [14, 94, 58, 66, 53, 80, 66,  0,  0,  0],
       [66, 67, 53,  0,  0,  0,  0,  0,  0,  0],
       [25, 72, 94,  0,  0,  0,  0,  0,  0,  0],
       [84, 64,  0,  0,  0,  0,  0,  0,  0,  0],
       [16, 94, 72, 82,  0,  0,  0,  0,  0,  0],
       [72, 64, 12,  0,  0,  0,  0,  0,  0,  0],
       [12, 53, 35, 72, 64, 12,  0,  0,  0,  0],
       [66, 67, 53,  0,  0,  0,  0,  0,  0,  0],
       [25, 84, 66, 67, 12, 94, 72, 25, 72,  3],
       [58, 32,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 6, 94, 84, 66, 84, 80, 67,  0,  0,  0],
       [66, 94, 58, 58, 14, 80,  0,  0,  0,  0],
       [32, 94, 58, 

In [32]:
print(len(X_char), len(X_word))

47959 47959


In [33]:
X_word[0].shape

(75,)

In [34]:
X_word[0]

array([30255,  8891, 17398,  4757,  4452,  6736, 11223, 33068, 11702,
        2991, 13191,  3373,  1083,  2400, 13808,  2991, 28206,  8891,
       25801, 18434, 23872,  2335,  8365, 18078,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [35]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

In [38]:
# y_tr.shape
# np.array(y_tr).reshape(len(y_tr), max_len, 1).shape
y_tr

array([[ 9,  9,  9, ...,  0,  0,  0],
       [ 6,  9,  9, ...,  0,  0,  0],
       [ 6, 15,  9, ...,  0,  0,  0],
       ...,
       [ 6,  9,  9, ...,  0,  0,  0],
       [ 9,  9,  9, ...,  0,  0,  0],
       [ 9, 12,  9, ...,  0,  0,  0]], dtype=int32)

In [39]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-7rtm4065
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-7rtm4065
You should consider upgrading via the '/home/ubuntu/env/bin/python3.8 -m pip install --upgrade pip' command.


In [43]:
pip install tensorflow-addons

     |████████████████████████████████| 679 kB 20.1 MB/s eta 0:00:01
You should consider upgrading via the '/home/ubuntu/env/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [44]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras_contrib.layers import CRF
import tensorflow_addons as tfa

In [41]:
n_chars

98

In [ ]:
# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

model = Model([word_in, char_in], out)


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
model.summary()

In [49]:
# input and embedding for words

# model = Sequential()
# model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
# model.add(Dropout(0.5))
# model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
# model.add(TimeDistributed(Dense(n_tags, activation="relu")))
# crf_layer = CRF(n_tags)
# model.add(crf_layer)


# model_2 = Sequential()
# model_2.add(Embedding(input_dim=n_words + 2, output_dim=20, input_length=max_len, mask_zero=True))

In [54]:
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

crf_layer = CRF(n_tags + 1)
crf_outputs = crf_layer(out)

model_2 = Model([word_in, char_in], crf_outputs)

model_lstm = Model([word_in, char_in], main_lstm)


In [48]:
# # input and embedding for words

# # model = Sequential()
# # model.add(Embedding(input_dim=n_words+1, output_dim=200, input_length=max_len))
# # model.add(Dropout(0.5))
# # model.add(Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.1)))
# # model.add(TimeDistributed(Dense(n_tags, activation="relu")))
# # crf_layer = CRF(n_tags)
# # model.add(crf_layer)


# # model_2 = Sequential()
# # model_2.add(Embedding(input_dim=n_words + 2, output_dim=20, input_length=max_len, mask_zero=True))

# word_in = Input(shape=(max_len,))
# emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
#                      input_length=max_len, mask_zero=True)(word_in)

# # input and embeddings for characters
# char_in = Input(shape=(max_len, max_len_char,))
# emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
#                            input_length=max_len_char, mask_zero=True))(char_in)
# # character LSTM to get word encodings by characters
# char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
#                                 recurrent_dropout=0.5))(emb_char)

# # main LSTM
# x = concatenate([emb_word, char_enc])
# x = SpatialDropout1D(0.3)(x)
# main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
#                                recurrent_dropout=0.6))(x)
# out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

# crf_outputs = tfa.layers.CRF(n_tags + 1)(out)

# model_3 = Model([word_in, char_in], crf_outputs)


In [55]:
model_2.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
model_2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 75, 10)]     0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 75)]         0                                            
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, 75, 10, 10)   1000        input_8[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 75, 20)       703620      input_7[0][0]                    
____________________________________________________________________________________________

In [53]:
model_2.layers[7]

In [56]:
model_lstm.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 75, 10)]     0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 75)]         0                                            
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, 75, 10, 10)   1000        input_8[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 75, 20)       703620      input_7[0][0]                    
____________________________________________________________________________________________

In [ ]:
history = model.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)

In [ ]:
lstm_states = model_lstm.predict(X)

In [57]:
word_in = Input(shape=(max_len,)) # (None, 75, 100)
emb_word = Embedding(input_dim=n_tags + 1, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in) # (None, 75, 20)
x = concatenate([lstm_states, emb_word]) # (None, 75, 120)
hidden_layer_1 = TimeDistributed(Dense(30, activation="sigmoid"))(x)
output_layer = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(hidden_layer)

model_rel = Model(inputs=word_in, outputs=output_layer)

NameError: name 'lstm_states' is not defined

In [58]:
!pip install conllu

You should consider upgrading via the '/home/ubuntu/env/bin/python3.8 -m pip install --upgrade pip' command.


In [59]:
from conllu import parse_incr

In [61]:
data_file = open("train.txt", "r", encoding="utf-8")
token_list = []
for tokenlist in parse_incr(data_file):
#     print(tokenlist)
    token_list.append(tokenlist)

In [62]:
len(token_list)

1

In [65]:
"a [b c] d [e f g]".split("\\s+(?!\\[^\\[]*\\])")

['a [b c] d [e f g]']

In [66]:
import regex as re

In [67]:
s = "a [b c] d [e f g]"
re.findall(r"\[[^[]*]|\([^)]*\)|<[^>]*>|\S+|(?=\n)", s)


['a', '[b c]', 'd', '[e f g]']

In [83]:
with open('train.txt') as f:
    lines = f.readlines()

In [81]:
token_list = []
for index, line in enumerate(lines):
    line = line.strip()
    line_split = re.findall(r"\[[^[]*]|\([^)]*\)|<[^>]*>|\S+|(?=\n)", line)
    token_list.append(line_split)

In [72]:
len(token_list)

27714

In [82]:
token_list[0]

['#doc', '1024']

In [110]:
df = pd.DataFrame(columns=['Word Token Index', 'Sentence #', 'Word', 'Tag', 'Relationship', 'Relation Index'])
token_list = []
cur_row_index = 0
for line in lines:
    line = line.strip()
    line_split = re.findall(r"\[[^[]*]|\([^)]*\)|<[^>]*>|\S+|(?=\n)", line)
    if line_split and '#doc' in line_split[0]:
        current_sentence_index = line_split[1]
    else:
        sentence = "#Sentence " + current_sentence_index
        word_token_index = line_split[0]
        word = line_split[1]
        pos = line_split[2]
        relation = line_split[3]
        relation_index = line_split[4]
        df.loc[cur_row_index] = [word_token_index, sentence, word, pos, relation, relation_index]
        cur_row_index += 1

In [111]:
[1] + [1,2,3]

[1, 1, 2, 3]

In [112]:
len(df)

26804

In [113]:
len(df['Sentence #'].unique())

910

In [114]:
26804 + 910

27714

In [204]:
data = df
data = data.fillna(method="ffill")
data.tail(10)

,Word Token Index,Sentence #,Word,Tag,Relationship,Relation Index
26794,27,#Sentence 4051,within,O,['N'],[27]
26795,28,#Sentence 4051,the,O,['N'],[28]
26796,29,#Sentence 4051,time,O,['N'],[29]
26797,30,#Sentence 4051,limit,O,['N'],[30]
26798,31,#Sentence 4051,set,O,['N'],[31]
26799,32,#Sentence 4051,by,O,['N'],[32]
26800,33,#Sentence 4051,the,O,['N'],[33]
26801,34,#Sentence 4051,UR,O,['N'],[34]
26802,35,#Sentence 4051,agreement,O,['N'],[35]
26803,36,#Sentence 4051,.,O,['N'],[36]


In [205]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); 
n_words

6389

In [206]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); 
n_tags

9

In [207]:
tags

['I-Loc',
 'B-Other',
 'I-Peop',
 'B-Org',
 'B-Loc',
 'O',
 'I-Other',
 'B-Peop',
 'I-Org']

In [208]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [209]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

None


In [210]:
sentences = getter.sentences

In [211]:
sentences[:3]

[[('Israel', 'B-Loc'),
  ('television', 'O'),
  ('rejected', 'O'),
  ('a', 'O'),
  ('skit', 'O'),
  ('by', 'O'),
  ('comedian', 'O'),
  ('Tuvia', 'B-Peop'),
  ('Tzafir', 'I-Peop'),
  ('that', 'O'),
  ('attacked', 'O'),
  ('public', 'O'),
  ('apathy', 'O'),
  ('by', 'O'),
  ('depicting', 'O'),
  ('an', 'O'),
  ('Israeli', 'B-Other'),
  ('family', 'O'),
  ('watching', 'O'),
  ('TV', 'O'),
  ('while', 'O'),
  ('a', 'O'),
  ('fire', 'O'),
  ('raged', 'O'),
  ('outside', 'O'),
  ('.', 'O')],
 [('Sen.', 'O'),
  ('Jake', 'B-Peop'),
  ('Garn', 'I-Peop'),
  (',', 'O'),
  ('R-Utah', 'O'),
  (',', 'O'),
  ('flew', 'O'),
  ('on', 'O'),
  ('the', 'O'),
  ('shuttle', 'O'),
  ('in', 'O'),
  ('April', 'O'),
  ('1985', 'B-Other'),
  ('and', 'O'),
  ('Rep.', 'O'),
  ('Bill', 'B-Peop'),
  ('Nelson', 'I-Peop'),
  (',', 'O'),
  ('D-Fla.', 'O'),
  (',', 'O'),
  ('was', 'O'),
  ('aboard', 'O'),
  ('in', 'O'),
  ('January', 'O'),
  ('1986', 'B-Other'),
  (',', 'O'),
  ('at', 'O'),
  ('the', 'O'),
  ('invitati

In [212]:
check_max_len = 0
for sentence in sentences:
    split_len = len(sentence)
    if split_len > 0:
        check_max_len = split_len
print(check_max_len)

30


In [213]:
max_len = 60
max_len_char = 10

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [214]:
print(word2idx["Legend"], tag2idx['I-Other'])

4068 7


In [215]:
from keras.preprocessing.sequence import pad_sequences
X_word = [[word2idx[w[0]] for w in s] for s in sentences]

In [216]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0
X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

76


In [217]:
from keras.utils.np_utils import to_categorical
y = [to_categorical(i, num_classes=n_tags + 1) for i in y]

In [239]:
y[0]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 

In [218]:
from sklearn.model_selection import train_test_split
X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

In [219]:
(np.array(y_tr)).shape

(819, 60, 10)

In [230]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
# from tensorflow.keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D, Reshape
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

from keras_contrib.layers import CRF
import tensorflow_addons as tfa

from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from tensorflow_addons.layers import CRF as crf

In [221]:
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
'''
out = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# crf_layer = CRF(n_tags + 1)
crf_layer = crf(n_tags + 1)
crf_outputs = crf_layer(out)

model_ner = Model([word_in, char_in], crf_outputs)

model_lstm_ner = Model([word_in, char_in], main_lstm)

model_ner.compile(optimizer="adam", loss=crf_loss, metrics=[crf_accuracy])
model_ner.summary()

out = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# crf_layer = CRF(n_tags + 1)
crf_layer = crf(n_tags + 1)
crf_outputs = crf_layer(out)

model_ner = Model([word_in, char_in], crf_outputs)

model_lstm_ner = Model([word_in, char_in], main_lstm)

model_ner.compile(optimizer="adam", loss=crf_loss, metrics=[crf_accuracy])
model_ner.summary()
'''
out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

model_ner = Model([word_in, char_in], out)

model_lstm_ner = Model([word_in, char_in], main_lstm)

model_ner.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model_ner.summary()

Model: "model_28"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           [(None, 60, 10)]     0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           [(None, 60)]         0                                            
__________________________________________________________________________________________________
time_distributed_66 (TimeDistri (None, 60, 10, 10)   780         input_47[0][0]                   
__________________________________________________________________________________________________
embedding_45 (Embedding)        (None, 60, 20)       127820      input_46[0][0]                   
___________________________________________________________________________________________

In [222]:
model_lstm_ner.summary()

Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           [(None, 60, 10)]     0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           [(None, 60)]         0                                            
__________________________________________________________________________________________________
time_distributed_66 (TimeDistri (None, 60, 10, 10)   780         input_47[0][0]                   
__________________________________________________________________________________________________
embedding_45 (Embedding)        (None, 60, 20)       127820      input_46[0][0]                   
___________________________________________________________________________________________

In [223]:
history = model_ner.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, n_tags + 1),
                    batch_size=32, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
21/21 [==============================] - 19s 537ms/step - loss: 1.0747 - acc: 0.6260 - val_loss: 0.8863 - val_acc: 0.7658
Epoch 2/30
21/21 [==============================] - 8s 386ms/step - loss: 0.7008 - acc: 0.7607 - val_loss: 0.5466 - val_acc: 0.7658
Epoch 3/30
21/21 [==============================] - 8s 383ms/step - loss: 0.4987 - acc: 0.7594 - val_loss: 0.5369 - val_acc: 0.7658
Epoch 4/30
21/21 [==============================] - 8s 387ms/step - loss: 0.4986 - acc: 0.7663 - val_loss: 0.5267 - val_acc: 0.7658
Epoch 5/30
21/21 [==============================] - 8s 390ms/step - loss: 0.4764 - acc: 0.7651 - val_loss: 0.5228 - val_acc: 0.7658
Epoch 6/30
21/21 [==============================] - 8s 385ms/step - loss: 0.4790 - acc: 0.7628 - val_loss: 0.5123 - val_acc: 0.7658
Epoch 7/30
21/21 [==============================] - 8s 384ms/step - loss: 0.4690 - acc: 0.7610 - val_loss: 0.4943 - val_acc: 0.7658
Epoch 8/30
21/21 [==============================] - 8s 383ms/step - loss: 0

In [148]:
# history = model_ner.fit([X_word_tr, np.array(X_char_tr)], np.array(y_tr), batch_size=32, epochs=5,
#                     validation_split=0.1, verbose=1)

Epoch 1/5


AttributeError: in user code:

    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:771 train_step  *
        loss = self.compiled_loss(
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/compile_utils.py:201 __call__  *
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/losses.py:142 __call__  *
        losses = call_fn(y_true, y_pred)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/losses.py:246 call  *
        return ag_fn(y_true, y_pred, **self._fn_kwargs)
    /home/ubuntu/env/lib/python3.8/site-packages/keras_contrib/losses/crf_losses.py:54 crf_loss  *
        crf, idx = y_pred._keras_history[:2]
    /home/ubuntu/e

In [224]:
# lstm_states = model_lstm_ner.predict(X)

model_lstm_ner_embedding = model_lstm_ner.predict([X_word_te,
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char))])

In [225]:
model_lstm_ner_embedding.shape

(91, 60, 100)

In [226]:
len(X_word_te)

91

In [227]:
print(model_lstm_ner_embedding[0])

[[-0.00589526  0.31680754 -0.20386294 ...  0.6420005   0.68665564
  -0.6912282 ]
 [-0.01005644  0.6949768   0.39937344 ...  0.43215862  0.5289223
  -0.6781849 ]
 [-0.00495683  0.61388475  0.80835253 ...  0.20846473  0.17628011
  -0.6845299 ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [228]:
print(model_lstm_ner_embedding[1])

[[-0.00693452  0.43423432 -0.14973465 ...  0.84933037  0.8812934
  -0.10105264]
 [-0.01058438  0.651135    0.42325923 ...  0.84718037  0.854559
  -0.3419517 ]
 [-0.01190342  0.7418245   0.7834609  ...  0.8597586   0.8108471
  -0.48762506]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [277]:
class SentenceGetterMultiLabel(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t, r, i, rel) for w, t, r, i, rel in zip(s["Word"].values.tolist(),
                                                     s["Tag"].values.tolist(),
                                                     s['Relation Index'].values.tolist(),
                                                     s['Word Token Index'].values.tolist(),
                                                     s['Relationship'].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [278]:
getter_multilabel = SentenceGetterMultiLabel(data)
sentences_multi_label = getter_multilabel.sentences

In [279]:
sentences_multi_label[:3]

[[('Israel', 'B-Loc', '[0]', '0', "['N']"),
  ('television', 'O', '[1]', '1', "['N']"),
  ('rejected', 'O', '[2]', '2', "['N']"),
  ('a', 'O', '[3]', '3', "['N']"),
  ('skit', 'O', '[4]', '4', "['N']"),
  ('by', 'O', '[5]', '5', "['N']"),
  ('comedian', 'O', '[6]', '6', "['N']"),
  ('Tuvia', 'B-Peop', '[7]', '7', "['N']"),
  ('Tzafir', 'I-Peop', '[0]', '8', "['Live_In']"),
  ('that', 'O', '[9]', '9', "['N']"),
  ('attacked', 'O', '[10]', '10', "['N']"),
  ('public', 'O', '[11]', '11', "['N']"),
  ('apathy', 'O', '[12]', '12', "['N']"),
  ('by', 'O', '[13]', '13', "['N']"),
  ('depicting', 'O', '[14]', '14', "['N']"),
  ('an', 'O', '[15]', '15', "['N']"),
  ('Israeli', 'B-Other', '[16]', '16', "['N']"),
  ('family', 'O', '[17]', '17', "['N']"),
  ('watching', 'O', '[18]', '18', "['N']"),
  ('TV', 'O', '[19]', '19', "['N']"),
  ('while', 'O', '[20]', '20', "['N']"),
  ('a', 'O', '[21]', '21', "['N']"),
  ('fire', 'O', '[22]', '22', "['N']"),
  ('raged', 'O', '[23]', '23', "['N']"),
  ('o

In [280]:
import json
json.loads('[34, 36]')

[34, 36]

In [281]:
for sent_index, sentence in enumerate(sentences_multi_label):
    for index, tup in enumerate(sentence):
        relation_index = tup[2]
        relation_index_loaded = json.loads(relation_index)
        relation_word_list = []
        for word_ndex in relation_index_loaded:
            relation_word_list.append(sentence[word_ndex][0])
        relation_word_list_serialized = json.dumps(relation_word_list)
        sentences_multi_label[sent_index][index] += (relation_word_list_serialized,)
        sentences_multi_label[sent_index][index] += (relation_index_loaded,)

In [282]:
sentences_multi_label[:3]

[[('Israel', 'B-Loc', '[0]', '0', "['N']", '["Israel"]', [0]),
  ('television', 'O', '[1]', '1', "['N']", '["television"]', [1]),
  ('rejected', 'O', '[2]', '2', "['N']", '["rejected"]', [2]),
  ('a', 'O', '[3]', '3', "['N']", '["a"]', [3]),
  ('skit', 'O', '[4]', '4', "['N']", '["skit"]', [4]),
  ('by', 'O', '[5]', '5', "['N']", '["by"]', [5]),
  ('comedian', 'O', '[6]', '6', "['N']", '["comedian"]', [6]),
  ('Tuvia', 'B-Peop', '[7]', '7', "['N']", '["Tuvia"]', [7]),
  ('Tzafir', 'I-Peop', '[0]', '8', "['Live_In']", '["Israel"]', [0]),
  ('that', 'O', '[9]', '9', "['N']", '["that"]', [9]),
  ('attacked', 'O', '[10]', '10', "['N']", '["attacked"]', [10]),
  ('public', 'O', '[11]', '11', "['N']", '["public"]', [11]),
  ('apathy', 'O', '[12]', '12', "['N']", '["apathy"]', [12]),
  ('by', 'O', '[13]', '13', "['N']", '["by"]', [13]),
  ('depicting', 'O', '[14]', '14', "['N']", '["depicting"]', [14]),
  ('an', 'O', '[15]', '15', "['N']", '["an"]', [15]),
  ('Israeli', 'B-Other', '[16]', '16

In [327]:
check_max_len = 0
for sentence in sentences_multi_label:
    split_len = len(sentence)
    if split_len > check_max_len:
        check_max_len = split_len
print(check_max_len)

max_len = 120
max_len_char = 10

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

113


In [328]:
from keras.preprocessing.sequence import pad_sequences
X_word = [[word2idx[w[0]] for w in s] for s in sentences_multi_label]

In [329]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0
X_char = []
for sentence in sentences_multi_label:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

y_ner = [[tag2idx[w[1]] for w in s] for s in sentences]
y_ner = pad_sequences(maxlen=max_len, sequences=y_ner, value=tag2idx["PAD"], padding='post', truncating='post')

76


In [371]:
len(X_word)

910

In [400]:
X_word[0]

array([5812,  648, 3271, 3585, 4072, 2274, 2201, 5497, 2822, 1762,  851,
       3454, 6049, 2274, 1307, 4879, 6035, 3887, 1801, 4431,  392, 3585,
       5399, 4810, 4109,  344,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [399]:
len(X_word[0])

120

In [403]:
print(len(X_char), len(X_char[0]), len(X_char[0][0]))

910 120 10


In [332]:
y_ner[0]

array([5, 6, 6, 6, 6, 6, 6, 8, 3, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [333]:
from keras.utils.np_utils import to_categorical
y_ner = [to_categorical(i, num_classes=n_tags + 1) for i in y_ner]

In [336]:
y_ner[0][1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [337]:
y_rel = [[w[-1] for w in s] for s in sentences_multi_label]

In [338]:
y_rel[0]

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [0],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25]]

In [339]:
# manual post padding
for index, y_relation in enumerate(y_rel):
    y_rel_len = len(y_relation)
    pad_len = max_len - y_rel_len
    y_relation_padded = y_relation + [0] * pad_len
    y_rel[index] = y_relation_padded

In [340]:
len(y_rel[0])

120

In [341]:
import numpy as np
a = np.arange(4).reshape(2, 2)
print(a)
b = np.add.reduce(a)
print(b)

[[0 1]
 [2 3]]
[2 4]


In [362]:
y_multi_labels = []
for index, y_relation in enumerate(y_rel):
    y_multi_label = []
    for rel_indices in y_relation:
        if isinstance(rel_indices, list):
            np_array = np.array(rel_indices)
            one_hot_array = np.zeros((np_array.size, max_len))
            try:
                one_hot_array[np.arange(np_array.size),np_array] = 1
            except Exception as e:
                print(np_array)
            reduced_one_hot = np.add.reduce(one_hot_array)
            y_multi_label.append(list(reduced_one_hot))
        else:
            one_hot_array = np.zeros((max_len,))
            one_hot_array[max_len - 1] = 1
            y_multi_label.append(list(one_hot_array))
    y_multi_labels.append(y_multi_label)

In [363]:
len(y_multi_labels[0][8])

120

In [364]:
from sklearn.model_selection import train_test_split
X_word_tr, X_word_te, y_tr_ner, y_te_ner = train_test_split(X_word, y_ner, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y_ner, test_size=0.1, random_state=2018)

_, _, y_tr_rel, y_te_rel = train_test_split(X_word, y_multi_labels, test_size=0.1, random_state=2018) 

In [365]:
X_word_tr[0]

array([3956,   46, 1457, 4453,  711,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [366]:
print(y_tr_ner[0][0], y_tr_ner[0][1], y_tr_ner[0][2], y_tr_ner[0][3], y_tr_ner[0][4], y_tr_ner[0][5], y_tr_ner[0][6])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [367]:
print(y_tr_rel[0][4], y_tr_rel[0][5], y_tr_rel[0][6])

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [368]:
X_word_te[0]

array([4548, 5964, 3587,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [369]:
print(y_te_ner[0][0], y_te_ner[0][1], y_te_ner[0][2], y_te_ner[0][3], y_te_ner[0][4], y_te_ner[0][5], y_te_ner[0][6])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [370]:
print(y_te_rel[0][1], y_te_rel[0][2], y_te_rel[0][3], y_te_rel[0][4])

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [387]:
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
'''
out = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# crf_layer = CRF(n_tags + 1)
crf_layer = crf(n_tags + 1)
crf_outputs = crf_layer(out)

model_ner = Model([word_in, char_in], crf_outputs)

model_lstm_ner = Model([word_in, char_in], main_lstm)

model_ner.compile(optimizer="adam", loss=crf_loss, metrics=[crf_accuracy])
model_ner.summary()

out = TimeDistributed(Dense(50, activation="relu"))(main_lstm)

# crf_layer = CRF(n_tags + 1)
crf_layer = crf(n_tags + 1)
crf_outputs = crf_layer(out)

model_ner = Model([word_in, char_in], crf_outputs)

model_lstm_ner = Model([word_in, char_in], main_lstm)

model_ner.compile(optimizer="adam", loss=crf_loss, metrics=[crf_accuracy])
model_ner.summary()
'''
ner_outputs = TimeDistributed(Dense(n_tags + 1, activation="softmax", name='ner_outputs'))(main_lstm)

label_embedding_ner_tags = Embedding(input_dim=n_tags + 1, output_dim=10,
                     input_length=max_len, mask_zero=True)(ner_outputs) # max_len -> TimeDistributed steps = max_len

label_embedding_ner_tags_reshaped = Reshape((max_len, ((n_tags + 1) * 10)))(label_embedding_ner_tags) #10 - output_dim

concat_lstm_label_embeddings = concatenate([main_lstm, label_embedding_ner_tags_reshaped])
hidden_layer_1 = TimeDistributed(Dense(max_len + 20, activation="sigmoid"))(concat_lstm_label_embeddings) # (60 time stamps) * (120 embedding column vecs) => In each time step: (1 * 120) * (120 * 30) -> (1 * 30)
dep_parser_outputs = TimeDistributed(Dense(max_len, activation="sigmoid", name='dep_parser_outputs'))(hidden_layer_1) # (1 * 30) * (30 * num_tags+1) -> (1 * num_tags+1)

# model_rel = Model([word_in, char_in], [ner_out, relation_ship_outputs])
model_rel = Model(inputs=[word_in, char_in], outputs=[ner_outputs, dep_parser_outputs])


# model_rel.compile(loss={'ner_outputs': 'categorical_crossentropy', 'dep_parser_outputs': 'binary_crossentropy'}, 
#               optimizer='adam', metrics={'ner_outputs': 'accuracy', 'dep_parser_outputs': 'accuracy'})
model_rel.compile(loss=['categorical_crossentropy', 'binary_crossentropy'], 
              optimizer='adam', metrics=['accuracy'])
model_rel.summary()

Model: "model_40"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_85 (InputLayer)           [(None, 120, 10)]    0                                            
__________________________________________________________________________________________________
input_84 (InputLayer)           [(None, 120)]        0                                            
__________________________________________________________________________________________________
time_distributed_147 (TimeDistr (None, 120, 10, 10)  780         input_85[0][0]                   
__________________________________________________________________________________________________
embedding_101 (Embedding)       (None, 120, 20)      127820      input_84[0][0]                   
___________________________________________________________________________________________

In [383]:
history = model_rel.fit(x=[X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    y={'ner_outputs': np.array(y_tr_ner).reshape(len(y_tr_ner), max_len, n_tags + 1), 'dep_parser_outputs': np.array(y_tr_rel).reshape(len(y_tr_rel), max_len, max_len)},
                    batch_size=32, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30


ValueError: in user code:

    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:830 train_function  *
        return step_function(self, iterator)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:813 run_step  *
        outputs = model.train_step(data)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/training.py:771 train_step  *
        loss = self.compiled_loss(
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/compile_utils.py:180 __call__  *
        y_true = self._conform_to_outputs(y_pred, y_true)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/compile_utils.py:56 _conform_to_outputs  *
        struct = map_to_output_names(outputs, self._output_names, struct)
    /home/ubuntu/env/lib/python3.8/site-packages/keras/engine/compile_utils.py:649 map_to_output_names  *
        raise ValueError('Found unexpected keys that do not correspond '

    ValueError: Found unexpected keys that do not correspond t

In [389]:
history = model_rel.fit(x=[X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    y=[np.array(y_tr_ner).reshape(len(y_tr_ner), max_len, n_tags + 1), np.array(y_tr_rel).reshape(len(y_tr_rel), max_len, max_len)],
                    batch_size=32, epochs=51, validation_split=0.2, verbose=1)

Epoch 1/51
21/21 [==============================] - 16s 745ms/step - loss: 0.0553 - time_distributed_149_loss: 0.0455 - time_distributed_151_loss: 0.0099 - time_distributed_149_accuracy: 0.9454 - time_distributed_151_accuracy: 0.0622 - val_loss: 0.1463 - val_time_distributed_149_loss: 0.1351 - val_time_distributed_151_loss: 0.0112 - val_time_distributed_149_accuracy: 0.8422 - val_time_distributed_151_accuracy: 0.0568
Epoch 2/51
21/21 [==============================] - 16s 742ms/step - loss: 0.0525 - time_distributed_149_loss: 0.0426 - time_distributed_151_loss: 0.0099 - time_distributed_149_accuracy: 0.9484 - time_distributed_151_accuracy: 0.0574 - val_loss: 0.1426 - val_time_distributed_149_loss: 0.1314 - val_time_distributed_151_loss: 0.0112 - val_time_distributed_149_accuracy: 0.8588 - val_time_distributed_151_accuracy: 0.0522
Epoch 3/51
21/21 [==============================] - 16s 746ms/step - loss: 0.0494 - time_distributed_149_loss: 0.0395 - time_distributed_151_loss: 0.0099 - ti

In [390]:
history = model_rel.fit(x=[X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    y=[np.array(y_tr_ner).reshape(len(y_tr_ner), max_len, n_tags + 1), np.array(y_tr_rel).reshape(len(y_tr_rel), max_len, max_len)],
                    batch_size=32, epochs=100, validation_split=0.2, verbose=1)

Epoch 1/100
21/21 [==============================] - 16s 743ms/step - loss: 0.0172 - time_distributed_149_loss: 0.0077 - time_distributed_151_loss: 0.0094 - time_distributed_149_accuracy: 0.9910 - time_distributed_151_accuracy: 0.0895 - val_loss: 0.1578 - val_time_distributed_149_loss: 0.1471 - val_time_distributed_151_loss: 0.0107 - val_time_distributed_149_accuracy: 0.8861 - val_time_distributed_151_accuracy: 0.0912
Epoch 2/100
21/21 [==============================] - 16s 750ms/step - loss: 0.0170 - time_distributed_149_loss: 0.0076 - time_distributed_151_loss: 0.0094 - time_distributed_149_accuracy: 0.9904 - time_distributed_151_accuracy: 0.0922 - val_loss: 0.1595 - val_time_distributed_149_loss: 0.1488 - val_time_distributed_151_loss: 0.0107 - val_time_distributed_149_accuracy: 0.8863 - val_time_distributed_151_accuracy: 0.0910
Epoch 3/100
21/21 [==============================] - 16s 750ms/step - loss: 0.0167 - time_distributed_149_loss: 0.0072 - time_distributed_151_loss: 0.0094 -

In [391]:
history = model_rel.fit(x=[X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    y=[np.array(y_tr_ner).reshape(len(y_tr_ner), max_len, n_tags + 1), np.array(y_tr_rel).reshape(len(y_tr_rel), max_len, max_len)],
                    batch_size=32, epochs=30, validation_split=0.2, verbose=1)

Epoch 1/30
21/21 [==============================] - 16s 753ms/step - loss: 0.0090 - time_distributed_149_loss: 0.0015 - time_distributed_151_loss: 0.0076 - time_distributed_149_accuracy: 0.9989 - time_distributed_151_accuracy: 0.2130 - val_loss: 0.1980 - val_time_distributed_149_loss: 0.1893 - val_time_distributed_151_loss: 0.0087 - val_time_distributed_149_accuracy: 0.8872 - val_time_distributed_151_accuracy: 0.2702
Epoch 2/30
21/21 [==============================] - 16s 748ms/step - loss: 0.0093 - time_distributed_149_loss: 0.0017 - time_distributed_151_loss: 0.0076 - time_distributed_149_accuracy: 0.9983 - time_distributed_151_accuracy: 0.2069 - val_loss: 0.1970 - val_time_distributed_149_loss: 0.1884 - val_time_distributed_151_loss: 0.0087 - val_time_distributed_149_accuracy: 0.8874 - val_time_distributed_151_accuracy: 0.2532
Epoch 3/30
21/21 [==============================] - 16s 749ms/step - loss: 0.0092 - time_distributed_149_loss: 0.0016 - time_distributed_151_loss: 0.0076 - ti

In [396]:
model.save("joint_ner_rel_model.h5")

In [393]:
!pip install seqeval

     |████████████████████████████████| 43 kB 3.6 MB/s  eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=7bd99c372ea60242d31cae5c27d8c43e152320c8dde4dc03f0edb040e76315f0
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
You should consider upgrading via the '/home/ubuntu/env/bin/python3.8 -m pip install --upgrade pip' command.


In [394]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [406]:
print(len(X_word_te), len(X_char_te), len(X_word_te[0]), len(X_char_te[0]), len(X_char_te[0][0]))

91 91 120 120 10


In [408]:
ner_preds, dep_parser_preds = model_rel.predict([X_word_te, np.array(X_char_te).reshape((len(X_char_te), max_len, max_len_char))], batch_size=32, verbose=1)

3/3 [==============================] - 0s 75ms/step


In [410]:
ner_preds.shape

(91, 120, 10)

In [411]:
dep_parser_preds.shape

(91, 120, 120)

In [412]:
ner_preds, dep_parser_preds = model_rel.predict([X_word_te, np.array(X_char_te).reshape((len(X_char_te), max_len, max_len_char))], verbose=1)

3/3 [==============================] - 0s 76ms/step


In [427]:
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
for i in range(len(ner_preds)):
    pred_index = np.argmax(ner_preds[i], axis=-1)
    gt_index = np.argmax(y_te_ner[i], axis=-1)
    print(30 * "=")
    for w, t, pred in zip(X_word_te[i], gt_index, pred_index):
        if w != 0:
            print("{:15}: {:5} {}".format(idx2word[w], idx2tag[t], idx2tag[pred]))
    if i == 5:
        break

Word           ||True ||Pred
Tokyo          : B-Loc B-Loc
MAINICHI       : B-Org B-Org
SHIMBUN        : I-Org I-Org
BK2506112994   : O     O
Bangkok        : B-Loc B-Loc
BANGKOK        : B-Org B-Loc
POST           : I-Org O
in             : O     O
English        : O     O
25             : B-Other B-Other
Jun            : I-Other B-Other
94             : I-Other I-Other
pp             : O     O
1              : O     O
,              : O     O
3              : O     O
Dominique      : B-Peop O
Beuff          : I-Peop O
of             : O     O
the            : O     O
International  : B-Org B-Org
Committee      : I-Org I-Org
of             : I-Org O
the            : I-Org O
Red            : I-Org O
Cross          : I-Org O
in             : O     O
Khartoum       : B-Loc O
said           : O     O
he             : O     O
had            : O     O
reports        : O     O
of             : O     O
people         : O     O
dying          : O     O
at             : O     O
the            : 

In [415]:
len(p)

120

In [416]:
p[0]

5

In [417]:
y_te[0]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 

In [418]:
p

array([5, 4, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

In [419]:
q = np.argmax(y_te[0], axis=-1)

In [420]:
q

array([5, 4, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [437]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [464]:
print("{:15}||{:5}||{}".format("Word", "Relation GT", "Relation Pred"))
for i in range(len(dep_parser_preds)):
    print(30 * "=")
    for j in range(max_len):
#         rel_pred_indices = list(np.where(np.array(dep_parser_preds[i][j] > 0.5)[0])
        rel_pred_indices = [np.argmax(dep_parser_preds[i][j])]
        rel_gt_indices = list(np.where(np.array(y_te_rel[i][j]) == 1)[0])
        w = X_word_te[i][j]
        if w != 0:
            print("{:15}: {:5} {}".format(idx2word[w], json.dumps(rel_gt_indices, cls=NpEncoder), json.dumps(rel_pred_indices, cls=NpEncoder)))
    if i == 5:
        break

Word           ||Relation GT||Relation Pred
Tokyo          : [0]   [0]
MAINICHI       : [1]   [1]
SHIMBUN        : [0]   [1]
BK2506112994   : [0]   [0]
Bangkok        : [1]   [0]
BANGKOK        : [2]   [1]
POST           : [1]   [2]
in             : [4]   [3]
English        : [5]   [4]
25             : [6]   [6]
Jun            : [7]   [8]
94             : [8]   [10]
pp             : [9]   [11]
1              : [10]  [11]
,              : [11]  [11]
3              : [12]  [14]
Dominique      : [0]   [0]
Beuff          : [9, 11] [1]
of             : [2]   [2]
the            : [3]   [3]
International  : [4]   [4]
Committee      : [5]   [5]
of             : [6]   [6]
the            : [7]   [7]
Red            : [8]   [8]
Cross          : [11]  [10]
in             : [10]  [12]
Khartoum       : [11]  [12]
said           : [12]  [13]
he             : [13]  [15]
had            : [14]  [16]
reports        : [15]  [16]
of             : [16]  [16]
people         : [17]  [19]
dying          : [18] 

In [441]:
# np.where(np.array([[1, 0, 1]]) == 1, axis=-1)

In [430]:
list(np.where(np.array([1, 0, 1]) == 1)[0])

[0, 2]

In [431]:
list(np.where(np.array([1, 0, 1]) > 0.5)[0])

[0, 2]

In [436]:
json.dumps([5])

'[5]'

In [449]:
y_te_rel[0][0] == 1.0

True

In [451]:
np.where(np.array(y_te_rel[0][0]) == 1.0)

(array([0]),)

In [455]:
np.argmax(dep_parser_preds[i][j])

0

In [463]:
[int(np.argmax(dep_parser_preds[0][0]))]

[0]